In [ ]:
import os, sys, importlib

import geopandas as gpd
import pandas as pd

import shapely.ops
from shapely.geometry import Point, LineString


In [ ]:
inLines = os.path.join(os.getcwd(), "SampleData","Canals_sample","Canals_edit.shp")
unqID = "UID"
inL = gpd.read_file(inLines)
#inL = inL.to_crs({'init':'epsg:3857'}) #need to project to a CRS with metres management
inL.head()

In [ ]:
def generate_subset_line(allCoords, length):
    #Generate the 
    allPts = []
    for cIdx in range(0, len(allCoords)):
        cPoint = Point(allCoords[cIdx])
        allPts.append(cPoint)
        if cIdx > 0:
            curLine = LineString(allPts)
            if curLine.length > length:
                break
    return(curLine)

In [ ]:
start = []
end = []
riverNameColumn = unqID
inL['startPer'] = 0
inL['endPer'] = 0
for idx, row in inL.iterrows():
    x = row['geometry']
    if x.geom_type == "LineString":
        totalLength = x.length
        startLength = totalLength * 0.4
        endLength   = totalLength * 0.2
        allCoords = list(x.coords)

        startLen = generate_subset_line(allCoords, startLength)
        allCoords.reverse()
        endLen   = generate_subset_line(allCoords, endLength)

        start.append([row[riverNameColumn], startLen])
        end.append([row[riverNameColumn], endLen])
        
        inL['startPer'].loc[idx] = startLen.length/totalLength
        inL['endPer'].loc[idx] = endLen.length/totalLength
        
        print("%s - %s - %s - %s" % (row[riverNameColumn], x.length, startLen.length, endLen.length))
    else:
        FUBAR='FOFF'
startDF = pd.DataFrame(start, columns=["NAME", "GEOM"])
endDF = pd.DataFrame(end,  columns=["NAME", "GEOM"])


In [ ]:
from shapely.wkt import loads

startDF_geom =startDF['GEOM']
startDF = startDF.drop(['GEOM'], axis=1)
startGDF = gpd.GeoDataFrame(startDF, geometry=startDF_geom, crs=inL.crs)

endDF_geom =endDF['GEOM']
endDF = endDF.drop(['GEOM'], axis=1)
endGDF = gpd.GeoDataFrame(endDF, geometry=endDF_geom, crs=inL.crs)



In [ ]:
startDF.to_csv(os.path.join(os.getcwd(), "SampleData","Canals_sample","canal_starts.csv"))
endDF.to_csv(os.path.join(os.getcwd(), "SampleData","Canals_sample","canal_ends.csv"))
inL.to_csv(os.path.join(os.getcwd(), "SampleData","Canals_sample","canals_with_length.csv"))

startGDF.to_file(os.path.join(os.getcwd(), "SampleData","Canals_sample","canal_starts.shp"))
endGDF.to_file(os.path.join(os.getcwd(), "SampleData","Canals_sample","canal_ends.shp"))
inL.to_file(os.path.join(os.getcwd(), "SampleData","Canals_sample","canal_with_length.shp"))

# Buffering and clipping
For each start and end segment, the feature needs to be buffered, but the buffer needs to be limited to not cover anything covered by other canals

In [ ]:
startDF

In [ ]:
curShape.buffer(bufferDist,cap_style=2, join_style=2, mitre_limit=1.0)

In [ ]:
bufferDist = 1000 # metres
bufferSet = startDF # endDF

for bufferSet, outName in [[startDF, 'canal_start_buffered_clipped'],[endDF, 'canal_end_buffered_clipped']]:
    bufferSet['GEOM2'] = ''
    outFile = os.path.join(os.getcwd(), "SampleData","Canals_sample","%s.shp" % outName)
    for idx, row in bufferSet.iterrows():
        curName = row['NAME']
        curShape = row['GEOM']        
        bufferShape = curShape.buffer(bufferDist,cap_style=2, join_style=2, mitre_limit=1.0)

        #Select canals that intersect current buffered shape
        intersecting_canals = inL[inL.intersects(bufferShape)]
        if intersecting_canals.shape[0] > 1:
            intersecting_canals = intersecting_canals[intersecting_canals[riverNameColumn] != curName]        
            for selCanal in intersecting_canals['geometry']:
                xx = shapely.ops.split(bufferShape, selCanal)
                maxArea = 0            
                selArea = None
                for x in xx:
                    if x.area > maxArea:
                        maxArea = x.area
                        selArea = x
                bufferShape = selArea
        bufferSet['GEOM2'].iloc[idx] = bufferShape
    bufferSet_geom = bufferSet['GEOM2']
    bufferSetGDF = bufferSet.drop('GEOM2', axis=1)
    bufferSetGDF = bufferSetGDF.drop('GEOM', axis=1)
    bufferSetGDF = gpd.GeoDataFrame(bufferSetGDF, geometry=bufferSet_geom, crs=inL.crs)
    bufferSetGDF.to_file(outFile)
    #bufferSet.to_csv(outFile)

In [ ]:
bufferSetGDF